In [9]:
import os
import cv2
from glob import glob

# ==== Cấu hình ====
images_dir = "D:/yolo8_congTruong/SODA/images/val"
labels_dir = "D:/yolo8_congTruong/SODA/labels/val"
output_dir = "classification_dataset/val"
person_class_id = 0
helmet_class_id = 1

min_width, min_height = 50, 50  # kích thước tối thiểu

os.makedirs(f"{output_dir}/helmet", exist_ok=True)
os.makedirs(f"{output_dir}/no_helmet", exist_ok=True)

label_files = glob(os.path.join(labels_dir, "*.txt"))

for label_file in label_files:
    base_name = os.path.splitext(os.path.basename(label_file))[0]

    # Tìm ảnh
    img_path = None
    for ext in [".jpg", ".png", ".jpeg"]:
        candidate = os.path.join(images_dir, base_name + ext)
        if os.path.exists(candidate):
            img_path = candidate
            break

    if img_path is None:
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue
    h, w = img.shape[:2]

    with open(label_file, "r") as f:
        lines = f.readlines()

    person_boxes = []
    helmet_boxes = []

    # Đọc tất cả bbox
    for line in lines:
        class_id, x_center, y_center, bw, bh = map(float, line.split())
        x1 = int((x_center - bw / 2) * w)
        y1 = int((y_center - bh / 2) * h)
        x2 = int((x_center + bw / 2) * w)
        y2 = int((y_center + bh / 2) * h)

        if int(class_id) == person_class_id:
            person_boxes.append((x1, y1, x2, y2))
        elif int(class_id) == helmet_class_id:
            helmet_boxes.append((x1, y1, x2, y2))

    # Xử lý từng người
    for i, (px1, py1, px2, py2) in enumerate(person_boxes):
        pw, ph = px2 - px1, py2 - py1
        if pw < min_width or ph < min_height:
            continue  # bỏ ảnh nhỏ

        has_helmet = False
        for (hx1, hy1, hx2, hy2) in helmet_boxes:
            # Tính giao nhau
            inter_x1 = max(px1, hx1)
            inter_y1 = max(py1, hy1)
            inter_x2 = min(px2, hx2)
            inter_y2 = min(py2, hy2)

            if inter_x2 > inter_x1 and inter_y2 > inter_y1:
                inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
                helmet_area = (hx2 - hx1) * (hy2 - hy1)
                if helmet_area > 0 and inter_area / helmet_area > 0.3:
                    has_helmet = True
                    break

        # Crop ảnh người
        person_crop = img[max(0, py1):min(h, py2), max(0, px1):min(w, px2)]
        if person_crop.size == 0:
            continue

        # Lưu ảnh
        if has_helmet:
            save_path = f"{output_dir}/helmet/{base_name}_{i}.jpg"
        else:
            save_path = f"{output_dir}/no_helmet/{base_name}_{i}.jpg"

        cv2.imwrite(save_path, person_crop)

print("✅ Hoàn tất chuyển đổi dataset!")


✅ Hoàn tất chuyển đổi dataset!


In [6]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from vit_pytorch import ViT
from tqdm import tqdm
import os

# ==== Cấu hình ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
epochs = 70
num_classes = 2  # helmet, no_helmet
save_path = "vit_helmet_best.pth"

# ==== Augmentation & Preprocessing ====
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# ==== Load Dataset ====
train_dataset = datasets.ImageFolder(
    root="D:/yolo8_congTruong/outputs/classification_dataset/train",
    transform=transform
)
val_dataset = datasets.ImageFolder(
    root="D:/yolo8_congTruong/outputs/classification_dataset/val",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ==== Khởi tạo model ViT ====
model = ViT(
    image_size=256,
    patch_size=32,
    num_classes=num_classes,
    dim=512,
    depth=6,
    heads=8,
    mlp_dim=1024,
    dropout=0.1,
    emb_dropout=0.1
).to(device)

# ==== Loss & Optimizer ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# ==== Train Loop ====
best_val_acc = 0.0  # lưu best accuracy
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(train_loader):.4f} - Train Acc: {train_acc:.2f}%")

    # ==== Validation ====
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100. * correct / total
    print(f"Validation Acc: {val_acc:.2f}%\n")

    # ==== Lưu mô hình tốt nhất ====
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved best model at epoch {epoch+1} with Validation Acc: {val_acc:.2f}%")

print(f"🏆 Training complete! Best Validation Acc: {best_val_acc:.2f}% - Model saved to {save_path}")


Epoch 1/70: 100%|██████████| 231/231 [00:39<00:00,  5.85it/s]


Epoch [1/70] - Loss: 0.6686 - Train Acc: 61.43%
Validation Acc: 50.96%

✅ Saved best model at epoch 1 with Validation Acc: 50.96%


Epoch 2/70: 100%|██████████| 231/231 [00:28<00:00,  8.22it/s]


Epoch [2/70] - Loss: 0.5815 - Train Acc: 69.90%
Validation Acc: 70.74%

✅ Saved best model at epoch 2 with Validation Acc: 70.74%


Epoch 3/70: 100%|██████████| 231/231 [00:28<00:00,  8.14it/s]


...

Epoch [68/70] - Loss: 0.5250 - Train Acc: 73.92%
Validation Acc: 68.75%



Epoch 69/70: 100%|██████████| 231/231 [00:28<00:00,  7.99it/s]


Epoch [69/70] - Loss: 0.5274 - Train Acc: 73.89%
Validation Acc: 70.66%



Epoch 70/70: 100%|██████████| 231/231 [00:28<00:00,  8.02it/s]


Epoch [70/70] - Loss: 0.5248 - Train Acc: 74.32%
Validation Acc: 69.52%

🏆 Training complete! Best Validation Acc: 79.12% - Model saved to vit_helmet_best.pth
